# Werewolf Simulator
The following runs a simulation of the game Werewolf with multiple agents, it is built on top of and uses the Agentscope library, documentation can be found here https://doc.agentscope.io/, and the open source code that we extend on is here https://github.com/modelscope/agentscope/tree/main.
### Rules of Werewolf
- There are 4 roles, the Werewolves, Villagers, Witch, and Seer (Witch and Seer are on the Villager team)
- Each night:
    - The Werewolves discuss and vote on a player to eliminate
    - The Witch is told what player the wolves voted to eliminate, and is given the choice to use their potion of healing to reserruct the eliminated player, or use their poison to eliminate another player. Note that each power can only be used once in a game
    - The seer can pick any other player and find out what their role is (one player per night)
- After the events of the night, all surviving players discuss amongst themselves and vote on a player to eliminate
- Werewolves win if their numbers equal or exceed Villagers.
- Villagers win if all Werewolves are eliminated.

In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

from typing import Optional, Union, Sequence, Any, List
from functools import partial
import openai 
import faiss
import numpy as np
import json
import csv 
from datetime import datetime 
from zoneinfo import ZoneInfo

import numpy as np
from loguru import logger

from agentscope.parsers.json_object_parser import MarkdownJsonDictParser
from agentscope.parsers import ParserBase
from agentscope.message import Msg
from agentscope.msghub import msghub
from agentscope.agents import AgentBase
from agentscope.memory.temporary_memory import TemporaryMemory
from agentscope.pipelines.functional import sequentialpipeline
from agentscope.manager import ModelManager

import agentscope

from utils.werewolf_utils import (
    extract_name_and_id,
    n2s,
    set_parsers,
)

## Vector Store Implementations
Vector store classes with both uncompressed, and reflective functionalities

In [3]:
class ReflectiveVectorstoreMemory:
    """Reflective Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)
        self.summaries = []  # To store summaries

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()
        response = client.embeddings.create(
            input=text,
            model=self.embedding_model
        )
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        for msg in message:
            self.messages.append(msg.name + ": " + msg.content)
        
    def summarize_cycle(self, cycle_type: str = "day"):
        """
        Generate a summary of the conversation after a day/night cycle,
        and add it to the vector store.
        
        Args:
            cycle_type (str): The type of cycle ("day" or "night").
        """
        client = openai.OpenAI()
        
        # Combine all messages since the last summary
        history = "\n".join(self.messages)
        
        # Use the slow mind (gpt-4o) to summarize
        slow_mind_response = client.chat.completions.create(
            model="gpt-4o",  # Slow mind model
            messages=[
                {"role": "system", "content": f"Summarize the following chat history from the last {cycle_type}:"},
                {"role": "user", "content": history}, 
            ],
        )
        summary = slow_mind_response.choices[0].message.content

        # Embed the summary and add it to the vector store
        embedding = self._get_embedding(summary)
        self.index.add(np.array([embedding]))
        self.summaries.append(summary)

        # Clear messages for the next cycle (current implementation gets summary from the point after the last summary was created)
        self.messages.clear()

    def get_relevant_summaries(self, query: str, top_k: int = 1):
        """Retrieve the top-k most relevant summaries.
        
        Args:
            query (str): query to find similar summaries to.
            top_k (int): number of relevant summaries to retrieve.
        """
        query_embedding = self._get_embedding(query)
        distances, indices = self.index.search(np.array([query_embedding]), top_k)
        results = [
            self.summaries[idx] for idx in indices[0] if idx < len(self.summaries)
        ]
        return results
    
    
class VectorstoreMemory:
    """Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()
        response = client.embeddings.create(
            input=text,
            model=self.embedding_model
        )
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        # add embedding of message to vector store
        embeddings = [self._get_embedding(msg.name + ": " + msg.content) for msg in message]
        self.index.add(np.array(embeddings)) 
        self.messages.extend(message)

    def get_relevant_messages(self, query: str, top_k: int = 10):
        """Retrieve the top-k most relevant messages."""
        query_embedding = self._get_embedding(query)
        distances, indices = self.index.search(np.array([query_embedding]), top_k)
        results = [
            self.messages[idx] for idx in indices[0] if idx < len(self.messages)
        ]
        return results

## Custom Agents (edit here)
Below are implementations of a custom Agent utilitizing the Slow Mind Fast Mind Architecture with a reflective vector store, and an agent that generates responses based off a normal vector store search.

In [4]:
global game_log

class SlowFastMindAgent(AgentBase):
    """An agent that generates response using the Fast Mind Slow Mind Architecture.
    
    Includes a FAISS vectorstore for memory
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        fast_mind_model_config_name: str,
        slow_mind_model_config_name: str,
        reflect_before_vectorstore: bool = False,
        similarity_top_k: int = 1,
        openai_api_key: str = "",
    ) -> None:
        """Initialize the custom agent.

        Arguments:
            name (`str`):
                The name of the agent.
            sys_prompt (`Optional[str]`, defaults to `None`):
                The system prompt of the agent, which can be passed by args
                or hard-coded in the agent.
            fast_mind_model_config_name (`str`, defaults to None):
                The name of the model config, which is used to load fast mind 
                model from configuration.
            slow_mind_model_config_name (`str`, defaults to None):
                The name of the model config, which is used to load slow mind 
                model from configuration.
            max_retries (`Optional[int]`, defaults to `None`):
                The maximum number of retries when failed to parse the model
                output.      
            similarity_top_k (`int`, defaults to `None`):
                The top k most similar items in the vectorstore that will be
                used in the context to the model
        """  # noqa
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
        )
        
        super().__init__(name=name, sys_prompt=sys_prompt)
        openai.api_key = openai_api_key
        self.parser = None
        self.reflect_before_vectorstore = reflect_before_vectorstore
        self.similarity_top_k = similarity_top_k

        # Load the two models from Agentscope's ModelManager
        model_manager = ModelManager.get_instance()
        self.fast_mind_model = model_manager.get_model_by_config_name(fast_mind_model_config_name)
        self.slow_mind_model = model_manager.get_model_by_config_name(slow_mind_model_config_name)

        if reflect_before_vectorstore:
            self.memory = ReflectiveVectorstoreMemory()
        else:
            self.memory = VectorstoreMemory()

    def set_parser(self, parser: ParserBase) -> None:
        """Set response parser, which will provide 1) format instruction; 2)
        response parsing; 3) filtering fields when returning message, storing
        message in memory. So developers only need to change the
        parser, and the agent will work as expected.
        """
        self.parser = parser

    def _build_prompt(self, x: Optional[Msg]):
        """Helper to build a prompt for either fast or slow mind."""
        
        query = x.content if x else ""
        if self.reflect_before_vectorstore:
            if len(self.memory.summaries) > 0:
                relevant_summaries = self.memory.get_relevant_summaries(query=query, top_k=self.similarity_top_k)
                summary_context = "\n".join(relevant_summaries)
            else:
                summary_context = ""

            game_log.append(f"Summary context: {summary_context}")
            prompt = [
                Msg("system", self.sys_prompt, role="system"),
                Msg("system", f"Summary: {summary_context}", role="system"),
                Msg("system", self.parser.format_instruction, role="system"),
            ]
        else:
            relevant_msgs = self.memory.get_relevant_messages(query, self.similarity_top_k)
            prompt = [
                Msg("system", self.sys_prompt, role="system"),
                relevant_msgs or x,
                Msg("system", self.parser.format_instruction, role="system"),
            ]
            game_log.append(f"Relevant messages: {'\n'.join([msg.content for msg in relevant_msgs])}")
        return prompt

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """Reply function of the agent.
        Processes the input data, generates a prompt using the current
        dialogue memory and system prompt, and invokes the language
        model to produce a response. The response is then formatted
        and added to the dialogue memory.

        Args:
            x (`Optional[Union[Msg, Sequence[Msg]]]`, defaults to `None`):
                The input message(s) to the agent, which also can be omitted if
                the agent doesn't need any input.


        Returns:
            `Msg`: The output message generated by the agent.

        Raises:
            `json.decoder.JSONDecodeError`:
                If the response from the language model is not valid JSON,
                it defaults to treating the response as plain text.
        """     
        if x is not None:
            self.memory.add_message(message=x)

        prompt = self._build_prompt(x)
        # format & call fast_mind
        model_prompt = self.fast_mind_model.format(*prompt)
        raw_response = self.fast_mind_model(model_prompt)

        self.speak(raw_response.stream or raw_response.text)
        parsed = self.parser.parse(raw_response)
        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(parsed.parsed),
        )
        self.memory.add_message(msg)
        return msg
    
    def fast_slow_mind(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """Fast Mind Slow Mind decision making function of the agent.

        Args:
            x (`Optional[Union[Msg, Sequence[Msg]]]`, defaults to `None`):
                The input message(s) to the agent, which also can be omitted if
                the agent doesn't need any input.


        Returns:
            `Msg`: The output message generated by the agent.

        Raises:
            `json.decoder.JSONDecodeError`:
                If the response from the language model is not valid JSON,
                it defaults to treating the response as plain text.
        """     
        if x is not None:
            self.memory.add_message(message=x)

        prompt = self._build_prompt(x)
        model_prompt = self.fast_mind_model.format(*prompt)
        fast_resp = self.fast_mind_model(model_prompt)

        # parse out 'uncertainty_score'
        try:
            data = json.loads(fast_resp.text.replace("```json", "").replace("```", "").strip())
            uncertainty = float(data.get("uncertainty_score", 0.0))
        except:
            uncertainty = 1.0

        if uncertainty >= 0.6:
            game_log.append("Regular agent is not satisfied with their certainty. Applying slow mind model...")
            # Switch to slow mind
            slow_prompt = self._build_prompt(x)
            slow_model_prompt = self.slow_mind_model.format(*slow_prompt)
            raw_response = self.slow_mind_model(slow_model_prompt)
        else:
            raw_response = fast_resp

        self.speak(raw_response.stream or raw_response.text)
        parsed = self.parser.parse(raw_response)
        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(parsed.parsed),
        )
        self.memory.add_message(msg)
        return msg
    
    def observe(self, x: Union[Msg, Sequence[Msg]]) -> None:
        """Observe the input, store it in memory without response to it.

        Args:
            x (`Union[Msg, Sequence[Msg]]`):
                The input message to be recorded in memory.
        """
        if x is not None:
            self.memory.add_message(message=x)
            
    def summarize_cycle(self, cycle_type: str = "day"):
        """
        Generate a summary at the end of a day or night cycle.
        
        Args:
            cycle_type (str): The type of cycle ("day" or "night").
        """
        if self.reflect_before_vectorstore:
            self.memory.summarize_cycle(cycle_type=cycle_type)
        
        
        
class NormalAgent(AgentBase):
    """Agent with normal uncompressed VectorStore,
    no other underlying decision making architecture.
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        max_retries: Optional[int] = 3,
        similarity_top_k: int = None,
        openai_api_key: str = "",
    ) -> None:
        """Initialize the dict dialog agent.

        Arguments:
            name (`str`):
                The name of the agent.
            sys_prompt (`Optional[str]`, defaults to `None`):
                The system prompt of the agent, which can be passed by args
                or hard-coded in the agent.
            model_config_name (`str`, defaults to None):
                The name of the model config, which is used to load model from
                configuration.
            max_retries (`Optional[int]`, defaults to `None`):
                The maximum number of retries when failed to parse the model
                output.
        """  # noqa
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )

        self.parser = None
        self.max_retries = max_retries
        openai.api_key = openai_api_key  # Set OpenAI API key
        self.memory = VectorstoreMemory()  # Use FAISS vector store for memory
        self.similarity_top_k = similarity_top_k

    def set_parser(self, parser: ParserBase) -> None:
        """Set response parser, which will provide 1) format instruction; 2)
        response parsing; 3) filtering fields when returning message, storing
        message in memory. So developers only need to change the
        parser, and the agent will work as expected.
        """
        self.parser = parser

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """Reply function of the agent.
        Processes the input data, generates a prompt using the current
        dialogue memory and system prompt, and invokes the language
        model to produce a response. The response is then formatted
        and added to the dialogue memory.

        Args:
            x (`Optional[Union[Msg, Sequence[Msg]]]`, defaults to `None`):
                The input message(s) to the agent, which also can be omitted if
                the agent doesn't need any input.


        Returns:
            `Msg`: The output message generated by the agent.

        Raises:
            `json.decoder.JSONDecodeError`:
                If the response from the language model is not valid JSON,
                it defaults to treating the response as plain text.
        """     
        # Add the input message to memory
        if x is not None:
            self.memory.add_message(message=x)
            
        query = x.content if x is not None else ""

        # prepare prompt with retrieved messages similar to input message
        prompt = self.model.format(
            Msg("system", self.sys_prompt, role="system"),
            self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)
            or x,  # type: ignore[arg-type]
            Msg("system", self.parser.format_instruction, "system"),
        )

        # call llm
        raw_response = self.model(prompt)

        self.speak(raw_response.stream or raw_response.text)

        # Parsing the raw response
        res = self.parser.parse(raw_response)

        msg = Msg(
            self.name,
            content=self.parser.to_content(res.parsed),
            role="assistant",
            metadata=self.parser.to_metadata(res.parsed),
        )
        
        # Save the response to memory
        self.memory.add_message(message=msg)

        return msg
    
    def observe(self, x: Union[Msg, Sequence[Msg]]) -> None:
        """Observe the input, store it in memory without response to it.

        Args:
            x (`Union[Msg, Sequence[Msg]]`):
                The input message to be recorded in memory.
        """
        if x is not None:
            self.memory.add_message(message=x)
            
            
# Register custom classes
agentscope.agents.SlowFastMindAgent = SlowFastMindAgent
agentscope.agents.NormalAgent = NormalAgent

## Prompts and Parser (edit minimally)
Below are the prompts that control and run the Werwolf game. They are implemented using the built in AgentScope MarkdownJsonDictParser which allows us to generate responses in a dictionary format that is compatible with our game. Currently structured to obtain both the private thoughts of an Agent which remain to themselves and showcase their reasoning, and the words that they speak to other agents.

Since these prompts control the game logic itself, we should try to limit the prompt edits here to a minimum, but we can make edits to the parser to control the reasoning process a particular role goes through before making a decision, eg. instead of a thought field, we ask it to make ask it to provide a reason to vote for a particular agent, and a reason to not vote for a particular agent, then ask it to make a final decision based off those fields. Fields other than "thought" however should not be touched as they play a critical role to the control flow of the game.

More details on the AgentScope parsers can be found here https://doc.agentscope.io/build_tutorial/structured_output.html

In [5]:
class Prompts:
    """Prompts for werewolf game"""

    to_wolves = (
        "{}, if you are the only werewolf, eliminate a player. Otherwise, "
        "discuss with your teammates and reach an agreement."
    )

    wolves_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "finish_discussion": "whether the discussion reached an agreement or not (true/false)",
        },
        required_keys=["thought", "speak", "finish_discussion"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["finish_discussion"],
    )

    to_wolves_vote = "Which player do you vote to kill?"

    # Adding uncertainty score for Fast Mind Slow Mind Architecture
    wolves_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
            "uncertainty_score": "score between 0-1 on how uncertain you are about your vote, 0 if your completely certain, 1 if your completely uncertain",
        },
        required_keys=["thought", "vote", "uncertainty_score"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_wolves_res = "The player with the most votes is {}."

    to_witch_resurrect = (
        "{witch_name}, you're the witch. Tonight {dead_name} is eliminated. "
        "Would you like to resurrect {dead_name}?"
    )

    to_witch_resurrect_no = "The witch has chosen not to resurrect the player."
    to_witch_resurrect_yes = "The witch has chosen to resurrect the player."

    witch_resurrect_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "whether to resurrect the player and the reason",
            "resurrect": "whether to resurrect the player or not (true/false)",
        },
        required_keys=["thought", "speak", "resurrect"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["resurrect"],
    )

    to_witch_poison = (
        "Would you like to eliminate one player? If yes, "
        "specify the player_name."
    )

    witch_poison_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "eliminate": "whether to eliminate a player or not (true/false)",
        },
        required_keys=["thought", "speak", "eliminate"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["eliminate"],
    )

    to_seer = (
        "{}, you're the seer. Which player in {} would you like to check "
        "tonight?"
    )

    seer_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "player_name",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    to_seer_result = "Okay, the role of {} is a {}."

    to_all_danger = (
        "The day is coming, all the players open your eyes. Last night, "
        "the following player(s) has been eliminated: {}."
    )

    to_all_peace = (
        "The day is coming, all the players open your eyes. Last night is "
        "peaceful, no player is eliminated."
    )

    to_all_discuss = (
        "Now the alive players are {}. Given the game rules and your role, "
        "based on the situation and the information you gain, to vote a "
        "player eliminated among alive players and to win the game, what do "
        "you want to say to others? You can decide whether to reveal your "
        "role."
    )

    survivors_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    # Adding uncertainty score for Fast Mind Slow Mind Architecture
    survivors_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
            "uncertainty_score": "score between 0-1 on how uncertain you are about your vote, 0 if your completely certain, 1 if your completely uncertain",

        },
        required_keys=["thought", "vote", "uncertainty_score"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_all_vote = (
        "Given the game rules and your role, based on the situation and the"
        " information you gain, to win the game, it's time to vote one player"
        " eliminated among the alive players. Which player do you vote to "
        "kill?"
    )

    to_all_res = "{} has been voted out."

    to_all_wolf_win = (
        "The werewolves have prevailed and taken over the village. Better "
        "luck next time!"
    )

    to_all_village_win = (
        "The game is over. The werewolves have been defeated, and the village "
        "is safe once again!"
    )

    to_all_continue = "The game goes on."

    
# Moderator message function
HostMsg = partial(Msg, name="Moderator", role="assistant", echo=True)

## Game Initialization (edit here)
To initialize the agents, you must define their parameters and settings in the config objects that are passed in for initialization. There is a model config, which defines the base foundational model being used, and an agent config, which defines each of the agents being used in the game, including which model their using, their name, and what type of Agent they are (based off the agent classes we defined earlier). 

Pay particular attention to the system prompt, this is what defines the rules of the game to the agent and gives them the role and what their responsibilities are, we could perhaps do some prompt engineering with that.

Also we can play around with the settings of the game, eg. max rounds, how many werewolves we have, etc. Just make sure to update the roles, witch, seer objects below accordingly.

In [6]:
# Holds game data 
game_log = []

# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

## Utility Functions (don't edit)
Functions to check and update game state throughout

In [7]:
def majority_vote(votes: list) -> Any:
    """Given a list of votes, return the name with the highest frequency."""
    votes_valid = [v for v in votes if v != "Abstain"]
    if not votes_valid:
        return "No Votes"
    unique_vals, counts = np.unique(votes_valid, return_counts=True)
    return unique_vals[np.argmax(counts)]

def update_alive_players(game_state: dict, survivors: list, wolves: list, dead_names):
    """
    Removes 'dead_names' from 'game_state["survivors"]', updates game_state["dead"],
    and returns updated survivors and wolves lists.
    """
    if not isinstance(dead_names, list):
        dead_names = [dead_names]
    for d in dead_names:
        if d in game_state["survivors"]:
            game_state["survivors"].remove(d)
        if d not in game_state["dead"]:
            game_state["dead"].append(d)

    # Rebuild survivors / wolves lists
    new_survivors = [s for s in survivors if s.name in game_state["survivors"]]
    new_wolves = [w for w in wolves if w.name in game_state["survivors"]]
    return new_survivors, new_wolves

def check_winning(game_state: dict, survivors: list, wolves: list, host: str) -> bool:
    """
    If #werewolves * 2 >= #survivors => werewolves instantly win.
    If all werewolves are dead => villagers instantly win.
    Otherwise => game continues.
    """
    if len(wolves) * 2 >= len(survivors):
        msg = Msg(host, Prompts.to_all_wolf_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "werewolves"
        logger.info(f"[Moderator] {Prompts.to_all_wolf_win}")
        return True
    if survivors and not wolves:
        msg = Msg(host, Prompts.to_all_village_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "villagers"
        logger.info(f"[Moderator] {Prompts.to_all_village_win}")
        return True
    return False

def generate_log_filepath(basepath: str, game_num: int) -> str:
    """Generate log file path with EST timestamp."""
    est_now = datetime.now(ZoneInfo("America/New_York"))
    iso_timestamp = est_now.isoformat(timespec='seconds').replace(":", "-")
    return f"{basepath}_{game_num}_{iso_timestamp}.log"

def save_logs_to_logfile(logs: list, filepath: str):
    """Write all log lines to a .log file. Creates directory if needed."""
    import os
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        for line in logs:
            f.write(line + "\n")
    print(f"Game log saved to {filepath}")

def save_results(win_rate_filepath: str, row: dict):
    """
    Append a row to a CSV file. If doesn't exist, create with headers.

    row = {
      "raw_log_filepath": <str>,
      "custom_agent_won": <bool>,
    }
    """
    import os
    import csv
    file_exists = os.path.isfile(win_rate_filepath)
    with open(win_rate_filepath, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["raw_log_filepath", "custom_agent_won"])
        if not file_exists:
            w.writeheader()
        w.writerow(row)
    print(f"Row added to {win_rate_filepath}: {row}")

def custom_agent_won(game_state: dict, alive_agents: list, wolf_agents: list):
    """
    Example "did our custom agent(s) or approach 'win'?" check.

    If game_state["endgame"], check if wolves >=. Otherwise, decide
    you 'won' if you're still alive. Adapt to your own logic.
    """
    if game_state["endgame"]:
        return (len(wolf_agents) * 2 >= len(alive_agents))
    # If not ended, you can define your own partial-win logic
    return True

## Running a Game 
Following is a function that game through the various Night and Day phases, taking different actions for each agent based on their roles. Multi agent functionality and communication is facilitated through the AgentScope Pipeline and MsgHub, more detailed documentation found here https://doc.agentscope.io/build_api/agentscope.pipelines.pipeline.html#module-agentscope.pipelines.pipeline and https://doc.agentscope.io/build_api/agentscope.msghub.html#module-agentscope.msghub

In [8]:
def run_game(
    game_num: int,
    max_days_per_game: int,
    raw_log_filepath: str,
    win_rate_filepath: str,
    reflect_before_vectorstore: bool,
    max_werewolf_discussion_round: int,
    wolves: list,
    seer,
    witch,
    roles,
    survivors,
    game_state
):
    """
    Runs a single game, storing logs in game_log. This example merges both
    day/night flow, werewolf talk, witch usage, seer usage, etc. Then
    saves logs and results.
    """
    global game_log
    game_log = []

    for day_i in range(1, max_days_per_game + 1):
        # 1) Night Phase: Werewolves discussion
        hint = HostMsg(content=Prompts.to_wolves.format(n2s(wolves)))
        game_log.append(f"Moderator: {hint.content}")
        with msghub(wolves, announcement=hint) as hub:
            set_parsers(wolves, Prompts.wolves_discuss_parser)
            for r in range(max_werewolf_discussion_round):
                x = sequentialpipeline(wolves)
                game_log.append(f"Werewolves discussion: {x.content}")
                if x.metadata.get("finish_discussion", False):
                    break
            # Then vote
            set_parsers(wolves, Prompts.wolves_vote_parser)
            hint_vote = HostMsg(content=Prompts.to_wolves_vote)
            game_log.append(f"Moderator: {hint_vote.content}")
            votes = [extract_name_and_id(wolf.fast_slow_mind(hint_vote).content)[0] for wolf in wolves]
            voted_out = majority_vote(votes)
            dead_player = [voted_out]
            game_log.append(f"Werewolves voted to eliminate: {voted_out}")
            hub.broadcast(HostMsg(content=Prompts.to_wolves_res.format(voted_out)))

        # witch decision night   
        healing_used_tonight = False
        if witch in survivors:
            if not game_state["witch_healing_used"]:
                hint = HostMsg(
                    content=Prompts.to_witch_resurrect.format_map(
                        {
                            "witch_name": witch.name,
                            "dead_name": dead_player[0],
                        },
                    ),
                )
                game_log.append(f"Moderator: {hint.content}")
                set_parsers(witch, Prompts.witch_resurrect_parser)
                
                # Capture the witch's resurrection response and log it
                resurrection_response = witch(hint)
                game_log.append(f"Witch resurrection response: {resurrection_response.content}")

                if resurrection_response.metadata.get("resurrect", False):
                    healing_used_tonight = True
                    dead_player.pop()
                    game_state["witch_healing_used"] = True
                    game_log.append("Moderator: The witch has chosen to resurrect the player.")
                    HostMsg(content=Prompts.to_witch_resurrect_yes)
                else:
                    game_log.append("Moderator: The witch has chosen NOT to resurrect the player.")
                    HostMsg(content=Prompts.to_witch_resurrect_no)

            if not game_state["witch_poison_used"] and not healing_used_tonight:
                set_parsers(witch, Prompts.witch_poison_parser)
                
                # Capture the witch's poison response and log it
                poison_response = witch(HostMsg(content=Prompts.to_witch_poison))
                game_log.append(f"Witch poison response: {poison_response.content}")

                if poison_response.metadata.get("eliminate", False):
                    target_player = extract_name_and_id(poison_response.content)[0]
                    dead_player.append(target_player)
                    game_state["witch_poison_used"] = True
                    game_log.append(f"Moderator: The witch has chosen to poison {target_player}.")


        # 3) Seer checks a role
        if seer in survivors:
            seer_hint = HostMsg(content=Prompts.to_seer.format(seer.name, n2s(survivors)))
            game_log.append(f"Moderator: {seer_hint.content}")
            set_parsers(seer, Prompts.seer_parser)
            x = seer.reply(seer_hint)
            game_log.append(f"Seer response: {x.content}")
            pl, idx = extract_name_and_id(x.content)
            role_name = roles[idx] if idx < len(roles) else "villager"
            # Let seer quietly observe the result
            role_msg = HostMsg(content=Prompts.to_seer_result.format(pl, role_name))
            game_log.append(f"Moderator: {role_msg.content}")
            seer.observe(role_msg)

        # 4) Update survivors after night
        survivors, wolves = update_alive_players(game_state, survivors, wolves, dead_player)
        if check_winning(game_state, survivors, wolves, "Moderator"):
            game_log.append("Moderator: Game ended after night.")
            break

        # If reflecting, do a night reflection
        if reflect_before_vectorstore:
            for w in wolves:
                w.summarize_cycle("night")

        # 5) Daytime announcements
        if dead_player:
            game_log.append(f"Moderator: {Prompts.to_all_danger.format(n2s(dead_player))}")
        else:
            game_log.append(f"Moderator: {Prompts.to_all_peace}")

        # 6) Daytime discussion
        discuss_hints = [
            HostMsg(content=Prompts.to_all_danger.format(n2s(dead_player))) if dead_player else HostMsg(content=Prompts.to_all_peace),
            HostMsg(content=Prompts.to_all_discuss.format(n2s(survivors)))
        ]
        with msghub(survivors, announcement=discuss_hints) as hub:
            set_parsers(survivors, Prompts.survivors_discuss_parser)
            discussion_out = sequentialpipeline(survivors)
            game_log.append(f"Survivors discussion: {discussion_out.content}")

            # Daytime vote
            set_parsers(survivors, Prompts.survivors_vote_parser)
            day_vote_hint = HostMsg(content=Prompts.to_all_vote)
            game_log.append(f"Moderator: {day_vote_hint.content}")
            # use fast mind slow mind for werewolf vote
            votes_day = [
                extract_name_and_id(wolf.fast_slow_mind(day_vote_hint).content)[0] for wolf in wolves
            ]

            votes_day.extend([
                extract_name_and_id(_.reply(day_vote_hint).content)[0] for _ in survivors if _ not in wolves
            ])
            day_result = majority_vote(votes_day)
            game_log.append(f"Daytime vote => {day_result}")
            hub.broadcast(HostMsg(content=Prompts.to_all_res.format(day_result)))

            survivors, wolves = update_alive_players(game_state, survivors, wolves, day_result)
            if check_winning(game_state, survivors, wolves, "Moderator"):
                game_log.append("Moderator: Game ended after day.")
                break

            # If reflecting, do a day reflection
            if reflect_before_vectorstore:
                for w in wolves:
                    w.summarize_cycle("day")

            hub.broadcast(HostMsg(content=Prompts.to_all_continue))

    # 7) Save logs
    updated_log_path = generate_log_filepath(raw_log_filepath, game_num)
    save_logs_to_logfile(game_log, updated_log_path)

    row = {
        "raw_log_filepath": updated_log_path,
        "custom_agent_won": custom_agent_won(
            game_state,
            survivors,
            wolves
        ),
    }
    save_results(win_rate_filepath, row)

## Main Entry: run_experiment()

In [9]:
def run_experiment(
    open_ai_key: str,
    raw_log_filepath: str,
    win_rate_filepath: str,
    max_days_per_game: int = 3,
    num_games: int = 2,
    reflect_before_vectorstore: bool = False,
    fast_mind_model: str = "gpt-4o-mini",
    slow_mind_model: str = "gpt-4o",
    max_werewolf_discussion_round: int = 3,
    similarity_top_k: int = 1
):
    """
    Example experiment runner that:
    1) Sets up model & agent configs.
    2) Initializes the game state.
    3) Plays multiple games.
    4) Logs to disk & CSV.

    :param open_ai_key: Your OpenAI API key from .env or direct string.
    :param raw_log_filepath: Base path for logs, e.g. "logs/game_raw"
    :param win_rate_filepath: CSV file to store results, e.g. "results.csv"
    :param max_days_per_game: How many day/night cycles to run per game
    :param num_games: How many times to run the simulation
    :param reflect_before_vectorstore: Toggle reflective memory
    :param primary_model: name of an Agentscope model config
    :param max_werewolf_discussion_round: how many times werewolves can pass the "mic"
    :param similarity_top_k: how many messages or summaries to retrieve from memory
    """

    load_dotenv()
    openai.api_key = open_ai_key  # set globally for our calls

    # 1) Define model configs and settings
    model_configs = [
        {
            "model_type": "openai_chat",
            "config_name": fast_mind_model,
            "model_name": fast_mind_model,
            "api_key": openai_key,
            "generate_args": {
                "temperature": 0.5
            }
        },
        {
            "model_type": "openai_chat",
            "config_name": slow_mind_model,
            "model_name": slow_mind_model,
            "api_key": openai_key,
            "generate_args": {
                "temperature": 0.5
            }
        }
    ]

    agent_configs = [
    {
        "class": "SlowFastMindAgent",
        "args": {
            "name": "Player1",
            "sys_prompt": "Act as a player in a werewolf game. You are Player1 and\nthere are totally 4 players, named Player1, Player2, Player3 and Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing werewolf in this game.\n",
            "fast_mind_model_config_name": fast_mind_model,
            "slow_mind_model_config_name": slow_mind_model,
            "reflect_before_vectorstore": reflect_before_vectorstore,
            "similarity_top_k": similarity_top_k,
            "openai_api_key": openai_key
        }
    },
    { 
        "class": "NormalAgent",
        "args": {
            "name": "Player2",
            "sys_prompt": "Act as a player in a werewolf game. You are Player2 and\nthere are totally 4 players, named Player1, Player2, Player3 and Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing villager in this game.\n",
            "model_config_name": fast_mind_model,
            "similarity_top_k": similarity_top_k,
            "openai_api_key": openai_key
        }
    },
    {
        "class": "NormalAgent",
        "args": {
            "name": "Player3",
            "sys_prompt": "Act as a player in a werewolf game. You are Player3 and\nthere are totally 4 players, named Player1, Player2, Player3 and Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing seer in this game.\n",
            "model_config_name": fast_mind_model,
            "similarity_top_k": similarity_top_k,
            "openai_api_key": openai_key
        }
    },
    {
        "class": "NormalAgent",
        "args": {
            "name": "Player4",
            "sys_prompt": "Act as a player in a werewolf game. You are Player4 and\nthere are totally 4 players, named Player1, Player2, Player3 and Player4.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing witch in this game.\n",
            "model_config_name": fast_mind_model,
            "similarity_top_k": similarity_top_k,
            "openai_api_key": openai_key
        }
    }
    ]

    # read model and agent configs, and initialize agents automatically
    survivors = agentscope.init(
        model_configs=model_configs,
        agent_configs=agent_configs,
        project="Werewolf",
    )
    # update these accordingly based off agent configs
    # currently 1 werewolf and 3 villagers
    roles = ["werewolf", "villager", "seer", "witch"]
    wolves, witch, seer = [survivors[0]], survivors[-1], survivors[-2]

    # initialize game state
    game_state = {
        "werewolves": [player.name for player in wolves],
        "villagers": [player.name for player in survivors if player not in wolves],
        "seer": [seer.name],
        "witch": [witch.name],
        "survivors": [player.name for player in survivors],
        "dead": [],
        "witch_healing_used": False,
        "witch_poison_used": False,
        "endgame": False,
        "winner": None
        
    }

    for game_num in range(1, num_games + 1):
        run_game(
            game_num,
            max_days_per_game,
            raw_log_filepath,
            win_rate_filepath,
            reflect_before_vectorstore,
            max_werewolf_discussion_round,
            wolves,
            seer,
            witch,
            roles,
            survivors,
            game_state
        )

In [ ]:
run_experiment(
    open_ai_key=openai_key,
    raw_log_filepath="slow_fast_logs_no_reflection/slow_fast_raw",
    reflect_before_vectorstore=False,
    win_rate_filepath="slow_fast_win_rate_no_reflection.txt",
    max_days_per_game=2,
    num_games=1,
    similarity_top_k=2
)

2025-02-10 23:22:57 | INFO     | agentscope.manager._model:load_model_configs:115 - Load configs for model wrapper: gpt-4o-mini, gpt-4o
2025-02-10 23:22:57 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-10 23:22:57 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for gpt-4o-mini: 'Model [gpt-4o-mini] not found in OPENAI_MAX_LENGTH. The last updated date is 20231212'
2025-02-10 23:22:57 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o]
2025-02-10 23:22:57 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for gpt-4o: 'Model [gpt-4o] not found in OPENAI_MAX_LENGTH. The last updated date is 20231212'
2025-02-10 23:22:57 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-10 23:22:57 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for